# Registration Methods 

This *Jupyter Notebook* is dedicated to the study of **Rigid Registration Methods**. 

The following study will be developed by the implementation of *Python* code for the application and visualization of the concepts learned with the use of *NumPy* and *Plotly* libraries. 

---

## Visual Setup

The plotting of the interactive graphic visuals will be done by a `Viewer` object based on the *Plotly*'s `Figure` object.

---

In [24]:
# Importing modules...
from modules.frame  import *
from modules.viewer import *
from modules.rigid  import *

## Validation Metrics

In each algorithm tryout, a **ground truth** rigid transformation will be randomly generated - composed of a rotation matrix and and a translation vector $R$ and $t$ (or in homogeneous representation, $H$).  

This rigid transformation will be applied to an original point-set $A$, generating the new point-set $B$, such that:

$$B = R\cdot A + t$$

or 

$$\tilde{B} = H \cdot \tilde{A}$$

Then, the proposed algorithm will calculate the estimated rotation matrix and translation vector $R_e$ and $t_e$ (or in homogeneous transformation, $H_e$). 

To verify how close these parameters are from the ground truth, both transformations, expressed in the homogeneous form, should satisfy as closely as possible the following expression:

$$||H-H_e|| \approx 0$$



---

In [25]:
# This function returns the norm of the difference between two matrices 
def dissimilarity(H, H_e):
    return np.linalg.norm(H-H_e)

## The Kabsch Algorithm

The Kabsch algorithm, also known as the Kabsch-Umeyama algorithm, named after Wolfgang Kabsch and Shinji Umeyama, is a method for calculating the optimal rotation matrix that minimizes the RMSD (root mean squared deviation) between two paired sets of points. It is useful for point-set registration in computer graphics, and in cheminformatics and bioinformatics to compare molecular and protein structures.

The following code is adapted and displayed from [here](http://nghiaho.com/?page_id=671).

---

In [26]:
# Find rotation and translation to align point-set A to point-set B
def kabsch(A, B):
    assert A.shape == B.shape

    # Find centroids and ensure they are 3x1
    A_c = np.mean(A, axis=1).reshape(-1, 1)
    B_c = np.mean(B, axis=1).reshape(-1, 1)

    # Centralize point-sets in origin
    A_0 = A - A_c
    B_0 = B - B_c

    H = A_0 @ np.transpose(B_0)

    # Find rotation using Singular Value Decomposition
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # Special reflection case
    if np.linalg.det(R) < 0: # The rotation matrix determinant should be +1
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    # Finding translation vector
    t = -R @ A_c + B_c

    return R, t

# Create graph viewer
viewer = Viewer('Kabsch Algorithm Implementation', graphical=True, size=2)

# Plot main frame
viewer.add_frame(Frame(), 'W')

# Original point-set
nPoints = 4
psA = square(position=np.array([1/2,1/2,0]).reshape(-1,1), size=1/10)

# Plot A
for i, A in enumerate(psA.T):
    viewer.add_point(A.reshape(-1,1), name=f'A{i}', color='red')

# Random rotation and translation
R, t = randR(), randt(1)

# Point-set B is a rotated and translated point-set A
psB = R @ psA + t

# Introduce noise in point-set B by changing the maximum noise distance delta
delta = 0.01
N = np.hstack(np.array([randt(delta) for _ in range(nPoints)])) # Noise matrix
psB += N

# Plot B
for i, B in enumerate(psB.T):
    viewer.add_point(B.reshape(-1,1), name=f'B{i}', color='blue')

# Calculated R and t by the Kabsch algorithm
R_e, t_e = kabsch(psA, psB)

# Estimated B
psB_e = R_e @ psA + t_e

# Plot estimated point-set B
for i, B_e in enumerate(psB_e.T):
    viewer.add_point(B_e.reshape(-1,1), name=f'Be{i}', color='green')
# If the algorithm is satisfactorily implemented, the green dataset (Estimated B) 
# must be as similar as possible as the blue dataset (Ground Truth B)

print('Norm of the difference: ', dissimilarity(join_homo(R, t), join_homo(R_e, t_e)))

viewer.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots

Norm of the difference:  0.04164055033482395
